# 数据源

## 相关技术

## 数据预处理

# 图谱构建

## 知识抽取

In [1]:
# -*- coding: utf-8 -*-
import re
file_name = 'D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点.txt'
out_file_name = 'D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点.csv'
dict_file_name = "D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点字典.txt"
pattern = r'[:|：]' 

# 得到当前层级数
def get_level(line):
    ret_level = -1
    lev = {
        1 : r'第[一|二|三|四|五|六]章',
        2 : r'[一|二|三|四|五|六|七|八|九]、',
        3 : r'（[一|二|三|四|五|六]）',
        4 : r'\d+、',
        5 : r'[⑴|⑵|⑶|⑷|⑸|⑹|⑺|⑻]'}
    for key in lev.keys():
        if re.match(lev[key],line):
            break
            
    if key:
        ret_level = key
    return ret_level, lev
details = []

with open(file_name,'r',encoding='utf-8') as f:
    current_lev_map = {}
    for line in f.readlines():
        line = line.strip('\n').strip()
        if len(line) > 0:
            result = re.split(pattern, line, maxsplit=1)
            level,lev_define = get_level(line)
            result = [x.strip('\n') for x in result]
#             print('level --> ',level, ' result --> ', result[0])
            result[0] = re.split(lev_define[level],result[0],maxsplit=1)[1]
            current_lev_map[level] = result[0]
            up_title = ''
            if level >= 2:
                up_title = current_lev_map[level-1]
            details.append([up_title.strip(), result[0].strip(), result[1].strip() if len(result) > 1 else '' ])
#             print('title:{}-->{}'.format(up_title, result))
import pandas as pd
import numpy as np
dataframe = pd.DataFrame(np.array(details),columns=['title','name','detail'])
dataframe.to_csv(out_file_name,index=False,sep=',',encoding='utf_8_sig')

with open(dict_file_name, 'w', encoding='utf-8') as f:
    content = dataframe.name.values.tolist()
    content.append('圆点')
    text = '\n'.join(content)
    f.writelines(text)
        
print('over !')

'''
清除所有数据
MATCH (n)
OPTIONAL MATCH (n)-[r]-()
DELETE n,r

注意生成的csv文件需保存到 neo4j 应用目录import 下
执行脚本：
LOAD CSV WITH HEADERS FROM "file:/math.csv" AS row
CREATE (n:Knowledge)
set n=row;
'''
    

over !


'\n注意生成的csv文件需保存到 neo4j 应用目录import 下\n执行脚本：\nLOAD CSV WITH HEADERS FROM "file:/math.csv" AS row\nCREATE (n:Knowledge)\nset n=row;\n'

## 关系抽取 

### 简单关系抽取（根据知识点文本内容抽取NEXT关系）

In [7]:

'''
1、根据文档结构建立PARTOF关系
执行脚本：
MATCH (a:Knowledge), (b:Knowledge) 
   WHERE a.name = b.title
CREATE (a)-[:INCLUDE]->(b) 
RETURN a,b 

2、根据CSV知识点内容中检索方式，建立USE关系
处理流程（极简流程）：
1） 对内容分词，然后通过分词去匹配， 注意（这里关系有可能是引用和使用两种，怎么样进行区分？？？）  分词工具包： https://github.com/lancopku/pkuseg-python
'''
import pandas as pd
import pkuseg
import numpy as np
from py2neo import Graph
dict_file_name = "D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点字典.txt"
file_name = 'D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点.csv'
wordseg = pkuseg.pkuseg(user_dict=dict_file_name)
dataframe = pd.read_csv(file_name)
graph = Graph(uri='bolt://127.0.0.1:7687',auth=('neo4j','654321')) 

kn_dict = None
with open(dict_file_name,'r',encoding='utf-8') as f:
    lines = f.readlines()
    kn_dict = [x.strip() for x in lines]
kn_dict.remove('圆点')
# print(kn_dict)
# 检测前置情况, 检测 cut text 是否在包含filter name
def is_pre(cut_text,filter_name):
    pre_kn = []
    cut_text = np.array(cut_text)
    for item in kn_dict:
        result = np.where(cut_text==item)
        if len(result[0]) > 0 and item != filter_name:
            pre_kn.append(item)
    return pre_kn

# 检测两个知识点是否有关系
def has_rel(k1, k2):
    result = False
    rdata = graph.run("MATCH (K:Knowledge {name:'%s'}) - [r] -> (P:Knowledge {name:'%s'}) RETURN r" % (k1,k2)).data()
    if (len(rdata) > 0):
        result = True
    return result

# 创建知识点NEXT关系 ， k2 -> NEXT --> k1, K1需要先掌握K2知识点
def create_next_rel(k1,k2):
    graph.run("MATCH (K1:Knowledge {name:'%s'}), (K2:Knowledge {name:'%s'}) CREATE (K2) - [:NEXT] -> (K1) return K1,K2" % (k1,k2))

for idx, row in dataframe.iterrows():
    detail = row['detail']
    if isinstance(detail,str) :
        text = wordseg.cut(detail)
        using_words = is_pre(text, row['name'])
        using_words = set(using_words)
        using_rel = [x for x in using_words if not has_rel(x,row['name'])]
#         print('kn -> {} pre kn : {} cut txt : {}'.format(row['name'], is_pre(text, row['name']), text))
        
        for item in using_rel:
#             print('kn -> {} next rel kn -> {}'.format(row['name'], item))
            create_next_rel(row['name'], item)
    
        




### 根据知识点成绩关联建立关系 

In [25]:
# 1、https://blog.csdn.net/luanpeng825485697/article/details/79808669 生成样本数据
# 2、https://blog.csdn.net/weixin_43060843/article/details/90269688?depth_1-utm_source=distribute.pc_relevant.none-task&utm_source=distribute.pc_relevant.none-task
#    python faker 模拟数据
# 3、https://cloud.tencent.com/developer/article/1576219 Python轻松实现统计学中重要的相关性分析
# 4、https://blog.csdn.net/sinat_29957455/article/details/79007269 python pandas 相关性分析
# 5、https://aic-fe.bnu.edu.cn/docs/20181205101832069569.pdf An Automatic Knowledge Graph Construction System forK-12 Education
'''
生成知识点成绩模拟数据
知识点： 
模拟需检测知识点：平行四边形计算公式(K0)
相关知识点：整数加法(K1)，整数减法(K2) --> 整数乘法(K3) --> 整数除法(K4)、常用的面积单位(K5)-->单位之间的换算(K6)、
无关知识点：货币单位换算(K7)、质量常用换算(K8)、时间单位换算(K9)、鸡兔问题(K10)

判断标准 P(Si -->Sj) AND P(^Sj --> ^Si)  : 关系为Sj为Si的前题，  P(Si -->Sj) 掌握了Si就等于掌握了Sj,  P(^Sj --> ^Si) : 没有掌握Sj就没有掌握Si
'''
import numpy as np
import pandas as pd

K1 = np.round(np.random.normal(90,5, 100),0)
K2 = np.round(np.random.normal(90,5, 100),0)
K3 = (K1 + K2) / 2
K4 = (K1 + K2) / 2
K5 = np.round(np.random.normal(90,5, 100),0)
K6 = K5 - 2
K7 = np.round(np.random.normal(90,5, 100),0)
K8 = np.round(np.random.normal(90,5, 100),0)
K9 = np.round(np.random.normal(90,5, 100),0)
K10 = np.round(np.random.normal(90,5, 100),0)
K0 = (K3 + K4 + K6) / 3

K = np.array([K0,K1,K2,K3,K4,K5,K6,K7,K8,K9,K10])
KP = pd.DataFrame(K.T, columns=['K0','K1','K2','K3','K4','K5','K6','K7','K8','K9','K10'])
print(KP.corr())


          四边        加法        减法        乘法        除法        面积        换算        货币        质量        时间        鸡兔
四边  1.000000  0.603799  0.620431  0.856930  0.856930  0.551827  0.551827  0.110345 -0.023628 -0.051363  0.053145
加法  0.603799  1.000000  0.020375  0.716635  0.716635  0.010857  0.010857  0.004721  0.032711 -0.038878 -0.113238
减法  0.620431  0.020375  1.000000  0.711905  0.711905  0.050748  0.050748  0.103199 -0.039354 -0.096052  0.075313
乘法  0.856930  0.716635  0.711905  1.000000  1.000000  0.043027  0.043027  0.075307 -0.004476 -0.094314 -0.027003
除法  0.856930  0.716635  0.711905  1.000000  1.000000  0.043027  0.043027  0.075307 -0.004476 -0.094314 -0.027003
面积  0.551827  0.010857  0.050748  0.043027  0.043027  1.000000  1.000000  0.092039 -0.038556  0.053040  0.146703
换算  0.551827  0.010857  0.050748  0.043027  0.043027  1.000000  1.000000  0.092039 -0.038556  0.053040  0.146703
货币  0.110345  0.004721  0.103199  0.075307  0.075307  0.092039  0.092039  1.000000  0.002507 -0.

In [19]:
??KP.cov

# 图谱扩展

## 增加资源节点 

## 增加习题节点 